![Egeria Logo](https://raw.githubusercontent.com/odpi/egeria/master/assets/img/ODPi_Egeria_Logo_color.png)

### ODPi Egeria Hands-On Lab
# Welcome to the Understanding Cohort Configuration Lab

## Introduction

ODPi Egeria is an open source project that provides open standards and implementation libraries to connect tools,
catalogs and platforms together so they can share information (called metadata) about data and the technology that supports it.

The ODPi Egeria repository services provide APIs for understanding the make up of the cohorts that an OMAG Server
is connected to. 
This hands-on lab steps through each of the repository services operations for understanding a cohort, providing a explaination and the code to call each operation.

## The Scenario

Gary Geeke is the IT Infrastructure leader at Coco Pharmaceuticals.  He has set up a number of OMAG Servers and
is validating they are operating correctly.

![Gary Geeke](https://raw.githubusercontent.com/odpi/data-governance/master/docs/coco-pharmaceuticals/personas/gary-geeke.png)

In this hands-on lab Gary is issuing queries to the repository services.  Gary's userId is `garygeeke`.

In [ ]:
import requests

adminUserId     = "garygeeke"

In the **Metadata Server Configuration**, gary configured servers for the OMAG Server Platforms shown in Figure 1:

![Figure 1](../images/coco-pharmaceuticals-systems-omag-server-platforms.png)
> **Figure 1:** Coco Pharmaceuticals' OMAG Server Platforms

Below are the host name and port number for the core, data lake and development platforms and the servers that are hosted on each:

In [ ]:
import os

corePlatformURL     = os.environ.get('corePlatformURL','http://localhost:8080') 
dataLakePlatformURL = os.environ.get('dataLakePlatformURL','http://localhost:8081') 
devPlatformURL      = os.environ.get('devPlatformURL','http://localhost:8082')

server1PlatformURL = dataLakePlatformURL
server1Name        = "cocoMDS1"

server2PlatformURL = corePlatformURL
server2Name        = "cocoMDS2"

server3PlatformURL = corePlatformURL
server3Name        = "cocoMDS3"

server4PlatformURL = dataLakePlatformURL
server4Name        = "cocoMDS4"

server5PlatformURL = corePlatformURL
server5Name        = "cocoMDS5"

server6PlatformURL = corePlatformURL
server6Name        = "cocoMDS6"

serverXPlatformURL = devPlatformURL
serverXName        = "cocoMDSx"

You can use these variables to issue the commands that follow to different servers.

Figure 2 shows which metadata servers belong to each cohort.

![Figure 2](../images/coco-pharmaceuticals-systems-metadata-servers.png)
> **Figure 2:** Membership of Coco Pharmaceuticals' cohorts

Below are the names of the three cohorts.

In [ ]:
cocoCohort = "cocoCohort"
devCohort  = "devCohort"
iotCohort  = "iotCohort"

## Querying a server's cohorts

The command below returns the list of cohorts that a server is connected to.


In [ ]:

import pprint
import json

def printResponse(response):
    prettyResponse = json.dumps(response.json(), indent=4)
    print(" ")
    print("Response: ")
    print(prettyResponse)
    print(" ")
    
def printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response):
    if response.status_code == 200:
        relatedHTTPCode = response.json().get('relatedHTTPCode')
        if relatedHTTPCode == 200:
            print("Unexpected response from server " + serverName)
            printResponse(response)
        else:
            exceptionErrorMessage = response.json().get('exceptionErrorMessage')
            exceptionSystemAction = response.json().get('exceptionSystemAction')
            exceptionUserAction   = response.json().get('exceptionUserAction')
            if exceptionErrorMessage != None:
                print(exceptionErrorMessage)
                print(" * " + exceptionSystemAction)
                print(" * " + exceptionUserAction)
            else:
                print("Unexpected response from server " + serverName)
                printResponse(response)
    else:
        print("Unexpected response from server platform " + serverPlatformName + " at " + serverPlatformURL)
        printResponse(response)
        

def queryServerCohorts(serverName, serverPlatformName, serverPlatformURL):
    try:
        metadataHighwayServicesURLcore =  '/servers/' + serverName + '/open-metadata/repository-services/users/' + adminUserId + '/metadata-highway'
        url = serverPlatformURL + metadataHighwayServicesURLcore + '/cohort-descriptions'
        response = requests.get(url)
        if response.status_code == 200:
            relatedHTTPCode = response.json().get('relatedHTTPCode')
            if relatedHTTPCode == 200:
                cohorts=response.json().get('cohorts')
                if cohorts == None:
                    print("Server " + serverName + " is not connected to any cohorts")
                else:
                    print("Server " + serverName + " is connected to the following cohorts:")
                    for x in range(len(cohorts)):
                        cohortName = cohorts[x].get('cohortName')
                        connectionStatus = cohorts[x].get('connectionStatus')
                        print (" * " + cohortName + " [" + connectionStatus + "]")
            else:
                printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
        else:
            printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
    except:
        print("Platform " + serverPlatformURL + " is unavailable")


serverName = "cocoMDS2"
serverPlatformURL = 'http://localhost:8080'
serverPlatformName = "platformName"

queryServerCohorts(serverName, serverPlatformName, serverPlatformURL)

----
## Querying local registration

The local registration describes the registration information that the metadata server
has shared with the cohorts it hass connected to.

In [ ]:

metadataHighwayServicesURLcore =  '/servers/' + serverName + '/open-metadata/repository-services/users/' + adminUserId + '/metadata-highway'

def printCohortMember(cohortMember, localRegistration):
    serverName = cohortMember.get('serverName')
    serverType = cohortMember.get('serverType')
    metadataCollectionId = cohortMember.get('metadataCollectionId')
    metadataCollectionName = cohortMember.get('metadataCollectionName')
    registrationTime = cohortMember.get('registrationTime')
    if localRegistration == True:
        print("Registration details for local " + serverType + " " + serverName)
    else:    
        print("Registration details for remote " + serverType + " " + serverName)
    print(" * Metadata collection id:   " + metadataCollectionId)
    print(" * Metadata collection name: " + metadataCollectionName)
    print(" * Registration time:        " + registrationTime)
    repositoryConnection = cohortMember.get('repositoryConnection')
    if repositoryConnection != None:
        endpoint = repositoryConnection.get('endpoint')
        if endpoint != None:
            address = endpoint.get('address')
            if address != None:
                print(" * URL for metadata queries: " + address)
            else:
                print(" * URL for metadata queries: null")
        else:
            print(" * URL for metadata queries: no endpoint")
    else:
        print(" * URL for metadata queries: not supported")


def printLocalRegistration(serverName, serverPlatformName, serverPlatformURL):
    try:
        metadataHighwayServicesURLcore =  '/servers/' + serverName + '/open-metadata/repository-services/users/' + adminUserId + '/metadata-highway'
        url = serverPlatformURL + metadataHighwayServicesURLcore + '/local-registration'
        response = requests.get(url)
        if response.status_code == 200:
            relatedHTTPCode = response.json().get('relatedHTTPCode')
            if relatedHTTPCode == 200:
                cohortMember = response.json().get('cohortMember')
                printCohortMember(cohortMember, True)
            else:
                printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
        else:
            printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
    except:
        print("Platform " + serverPlatformURL + " is unavailable")
        
        
printLocalRegistration(serverName, serverPlatformName, serverPlatformURL)

print (" ")
print ("Querying local registration for " + serverName + " ...")
url = platformURLroot + metadataHighwayServicesURLcore + '/local-registration'
print ("GET " + url)

response = requests.get(url)
if response.status_code == 200:
    cohortMember = response.json().get('cohortMember')
    if cohortMember == None:
        exceptionErrorMessage = response.json().get('exceptionErrorMessage')
        if exceptionErrorMessage != None:
            print(exceptionErrorMessage)
        else:
            print("Server " + serverName + " is not registered with any cohorts")
    else:
        registrationTime = cohortMember.get('registrationTime')
        if registrationTime == None:
            print("Server " + serverName + " has not registered with the cohort yet")
        else:
            print("Server " + serverName + " first registered with a cohort at " + registrationTime)
        repositoryConnection = cohortMember.get('repositoryConnection')
        if repositoryConnection != None:
            endpoint = repositoryConnection.get('endpoint')
            if endpoint != None:
                address = endpoint.get('address')
                if address != None:
                    print("Server " + serverName + " is advertising the following URL for metadata queries " + address)
                else:
                    print("Server " + serverName + " is not advertising a URL for metadata queries")
            else:
                print("Server " + serverName + " has no endpoint for metadata queries")
        else:
            print("Server " + serverName + " does not support metadata queries")
else:
    print("Cohort configuration is unavailable")
    
print (" ") 
prettyResponse = json.dumps(response.json(), indent=4)
print ("Response: ")
print (prettyResponse)
print (" ")

----
## Querying remote members



In [ ]:
def printRemoteRegistrations(serverName, cohortName, serverPlatformName, serverPlatformURL):
    try:
        metadataHighwayServicesURLcore =  '/servers/' + serverName + '/open-metadata/repository-services/users/' + adminUserId + '/metadata-highway'
        url = serverPlatformURL + metadataHighwayServicesURLcore + '/cohorts/' + cohortName + '/remote-members'
        response = requests.get(url)
        if response.status_code == 200:
            relatedHTTPCode = response.json().get('relatedHTTPCode')
            if relatedHTTPCode == 200:
                cohortMembers = response.json().get('cohortMembers')
                for x in range(len(cohorts)):
                    printCohortMember(cohortMembers[x], False)
            else:
                printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
        else:
            printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
    except:
        print("Platform " + serverPlatformURL + " is unavailable")
        
        
printRemoteRegistrations(serverName, "cocoCohort", serverPlatformName, serverPlatformURL)


print (" ")
print ("Querying remote members for " + serverName + " ...")
url = platformURLroot + metadataHighwayServicesURLcore + "/cohorts/cocoCohort/remote-members"
print ("GET " + url)

response = requests.get(url)

print (" ")
prettyResponse = json.dumps(response.json(), indent=4)
print ("Response: ")
print (prettyResponse)
print (" ")

----